In [2]:
import tensorflow as tf 

"""

on a fpga a normal pytorch or tensorflow model wont work until it is converted to a format that the fpga can understand
this is done using a tool called Vitis AI which is provided by Xilinx

"""

""" 
to know about hls 

what is hls?

HLS stands for High-Level Synthesis. It is a process that takes a high-level description of a design (usually in a language like C, C++, or SystemC) and converts it into a hardware description language (HDL) like VHDL or Verilog. This allows designers to work at a higher level of abstraction, making it easier to design complex systems without having to manually code in HDL.

HLS is particularly useful for FPGA design because it can significantly reduce the time and effort required to create and optimize hardware implementations. By using HLS tools, designers can focus on the algorithm and functionality of their design, while the tool takes care of the low-level details of the hardware implementation.

"""

In [ ]:
''' 
change of plans 
'''



In [2]:
import cv2 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten , Conv2D 
from keras.models import Sequential
import os 

2025-09-01 12:46:07.267665: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
os.listdir()

['archive(1)']

In [9]:
base_path="archive(1)/real_and_fake_face"

In [14]:
X=[]
y=[]

In [16]:
!ls archive\(1\)

]4;0;#0E1C21\]1;0;#0E1C21\]4;1;#8383FF\]4;2;#64DEE1\]4;3;#75FCDD\]4;4;#79B3C4\]4;5;#7AAEEA\]4;6;#81D7E1\]4;7;#CCDBD5\]4;8;#B1BCB5\]4;9;#BCB9FF\]4;10;#F2FFFF\]4;11;#FFFFFF\]4;12;#BFE1EA\]4;13;#C8DAFF\]4;14;#EDFDFF\]4;15;#B7EAFF\]10;#CCDBD5\]11;[100]#0E1C21\]12;#CCDBD5\]13;#CCDBD5\]17;#CCDBD5\]19;#0E1C21\]4;232;#CCDBD5\]4;256;#CCDBD5\]708;[100]#0E1C21\]11;#0E1C21\
 real_and_fake_face   real_and_fake_face_detection


In [22]:
import os
import cv2
import numpy as np
base_path="archive/real_and_fake_face"
X=[]
y=[]
train_real_dir=os.path.join(base_path,"training_real")
train_fake_dir=os.path.join(base_path,"training_fake")

IMG_SIZE=(128,128)

for filename in os.listdir(train_real_dir):
    img_path=os.path.join(train_real_dir,filename)
    img = cv2.imread(img_path) 
    if img is not None:
        img = cv2.resize(img,IMG_SIZE)
        X.append(img)
        y.append(1)
for filename in os.listdir(train_fake_dir):
    img_path=os.path.join(train_fake_dir,filename)
    img = cv2.imread(img_path) 
    if img is not None:
        img = cv2.resize(img,IMG_SIZE)
        X.append(img)
        y.append(0)

X=np.array(X)
y=np.array(y)
X=X/255.0

x_test=[]
y_test=[]

base_path="archive/real_and_fake_face_detection"
test_real_dir=os.path.join(base_path,"training_real")
test_fake_dir=os.path.join(base_path,"training_fake")



In [26]:
base_path="/home/waasi/waasi/python/kura_fpga/archive/real_and_fake_face_detection/real_and_fake_face"
test_real_dir=os.path.join(base_path,"training_real")
test_fake_dir=os.path.join(base_path,"training_fake")
IMG_SIZE=(128,128)
for filename in os.listdir(test_real_dir):
    img_path=os.path.join(test_real_dir,filename)
    img = cv2.imread(img_path) 
    if img is not None:
        img = cv2.resize(img,IMG_SIZE)
        x_test.append(img)
        y_test.append(1)
for filename in os.listdir(test_fake_dir):
    img_path=os.path.join(test_fake_dir,filename)
    img = cv2.imread(img_path) 
    if img is not None:
        img = cv2.resize(img,IMG_SIZE)
        x_test.append(img)
        y_test.append(0)

x_test=np.array(x_test)
y_test=np.array(y_test)
x_test=x_test/255.0

In [27]:
import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms


x_train_tensor = torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2)  # Change to (N, C, H, W)
y_train_tensor = torch.tensor(y, dtype=torch.long)  
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)  # Change to (N, C, H, W)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [28]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjusted for 128x128 input size
        self.fc2 = nn.Linear(128, 2)  # Binary classification
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)  # Flatten the tensor
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [31]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.7614622209221125
Epoch 2, Loss: 0.6852169428020716
Epoch 3, Loss: 0.6728378226980567
Epoch 4, Loss: 0.6680823666974902
Epoch 5, Loss: 0.6588665349408984
Epoch 6, Loss: 0.6463717725127935
Epoch 7, Loss: 0.6287095723673701
Epoch 8, Loss: 0.6169989723712206
Epoch 9, Loss: 0.5778119997121394
Epoch 10, Loss: 0.5471758833155036


In [32]:
# get the model accuracy
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()   

In [33]:
# print the accuracy
print(f"Accuracy: {100 * correct / total}%")

Accuracy: 75.01224889759922%


In [34]:
# how do i save the model
torch.save(model.state_dict(), "model.pth")

In [35]:
# use the model that i have saved
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [36]:
# i can now predict an external image
img_path="IMG_0234.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img,IMG_SIZE)
img = img/255.0
img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)  # Change to (1, C, H, W)
output = model(img_tensor)
_, predicted = torch.max(output.data, 1)
print(f"Predicted class: {predicted.item()}")  # 0 for fake, 1 for real 

Predicted class: 1


In [37]:
img_path="IMG_20240808_091916928_BURST000_COVER_TOP.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img,IMG_SIZE)
img = img/255.0
img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)  # Change to (1, C, H, W)
output = model(img_tensor)
_, predicted = torch.max(output.data, 1)
print(f"Predicted class: {predicted.item()}")  # 0 for fake, 1 for realtem()}")  

Predicted class: 1


In [38]:
img_path="woman_with_fake_boobs.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img,IMG_SIZE)
img = img/255.0
img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0,1).unsqueeze(0)  # Change to (1, C, H, W)
output = model(img_tensor)
_, predicted = torch.max(output.data, 1)
print(f"Predicted class: {predicted.item()}")  # 0 for fake, 1 for real

Predicted class: 1


In [44]:
img_path="zoro.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img,IMG_SIZE)
img = img/255.0
img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)  # Change to (1, C, H, W)
output = model(img_tensor)
_, predicted = torch.max(output.data, 1)
print(f"Predicted class: {predicted.item()}")  # 0 for fake, 1 for real

Predicted class: 0


In [ ]:
img_path=""